# MathLang DSL 入力テスト

このノートブックでは、MathLangのDSL（Domain Specific Language）形式の入力を解析し、実行するテストを行います。

## テスト内容
1. DSLパーサーの動作確認
2. Evaluatorによる実行確認
3. 結果の表示

In [1]:
# パスの設定
import sys
from pathlib import Path

project_root = Path().absolute().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"プロジェクトルート: {project_root}")

# 必要なモジュールのインポート
from core.parser import Parser
from core.evaluator import Evaluator
from core.core_runtime import CoreRuntime
from core.symbolic_engine import SymbolicEngine
from core.computation_engine import ComputationEngine
from core.validation_engine import ValidationEngine
from core.hint_engine import HintEngine

print("モジュールのインポート完了")

プロジェクトルート: /Users/chigenori/development/mathlang
モジュールのインポート完了


In [2]:
# ランタイムの初期化
symbolic_engine = SymbolicEngine()
computation_engine = ComputationEngine(symbolic_engine)
validation_engine = ValidationEngine(computation_engine)
hint_engine = HintEngine(computation_engine)

runtime = CoreRuntime(
    computation_engine=computation_engine,
    validation_engine=validation_engine,
    hint_engine=hint_engine
)

print("CoreRuntime 初期化完了")

CoreRuntime 初期化完了


## DSLの定義と実行

以下のセルでDSLテキストを定義し、パースして実行します。

In [3]:
def run_dsl(dsl_text):
    print("=" * 40)
    print("DSL入力:")
    print("-" * 40)
    print(dsl_text.strip())
    print("=" * 40)
    
    # %MathLang ヘッダーがあれば削除（パーサーが対応していない場合のため）
    lines = [line for line in dsl_text.splitlines() if not line.strip().startswith("%MathLang")]
    clean_dsl = "\n".join(lines)
    
    try:
        # パース
        print("\n1. パース中...")
        parser = Parser(clean_dsl)
        program = parser.parse()
        print("   ✓ パース成功")
        
        # 実行
        print("\n2. 実行中...")
        # 実行ごとにランタイムの状態をリセットする必要がある場合はここでリセット
        # runtime.reset() # もしあれば
        
        evaluator = Evaluator(program, runtime)
        success = evaluator.run()
        
        print(f"   {'✓ 実行成功' if success else '✗ 実行失敗'}")
        
    except Exception as e:
        print(f"\n✗ エラーが発生しました: {e}")
        import traceback
        traceback.print_exc()

In [6]:
# テストケース1: 正常な展開
dsl_case1 = """
%MathLang
problem: (x + 1)^2
step: (x + 1)(x + 1)
step: x(x + 1) + 1(x + 1)
step: x2 + x + x + 1
step: x^2 + 2x + 1
end: done
"""

run_dsl(dsl_case1)

DSL入力:
----------------------------------------
%MathLang
problem: (x + 1)^2
step: (x + 1)(x + 1)
step: x(x + 1) + 1(x + 1)
step: x2 + x + x + 1
step: x^2 + 2x + 1
end: done

1. パース中...
   ✓ パース成功

2. 実行中...
   ✓ 実行成功


In [5]:
# テストケース2: 途中計算の間違い
dsl_case2 = """
%MathLang
problem: 2*x + 3*x
step: 6*x  # 足し算なのに掛け算してしまった
end: 5*x
"""

run_dsl(dsl_case2)

DSL入力:
----------------------------------------
%MathLang
problem: 2*x + 3*x
step: 6*x  # 足し算なのに掛け算してしまった
end: 5*x

1. パース中...
   ✓ パース成功

2. 実行中...
   ✓ 実行成功
